# Exercise 3

## Setup

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql import functions as F

In [26]:
file_path = '../data/tour_occ_ninat.p.csv'

In [27]:
spark = SparkSession.builder.appName("Exercise 3").getOrCreate()

spark

## Data Loading

In [28]:
df = spark.read.csv(file_path, header=True, inferSchema=True)

df.show()

+--------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|       Country|     2006|     2007|     2008|     2009|     2010|     2011|     2012|     2013|     2014|     2015|
+--------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|       Belgium| 16039090| 16271311| 16360702| 15451017| 16169676| 16723867| 16432646| 16511721| 17068872| 18852087|
|      Bulgaria| 11944694| 12006786| 11791454|  9460922| 10547112| 12426723| 13451440| 14370426| 14077798| 13352281|
|Czech Republic| 20090348| 20610186| 19987022| 17746893| 18365947| 19424839| 21793985| 22144896| 22110112| 23286515|
|       Denmark|  9453026|  9327579|  8918197|  8299403|  8981992|  9491137|  9608124|  9914273| 10608119| 11171416|
|       Germany| 52947373| 54485379| 56239679| 54096574| 59658760| 63081467| 68161503| 71191942| 74805253| 78827773|
|       Estonia|  3020367|  2915456|  2932662|  2740696|  320372

### Flattening the DataFrame

We'll flatten the DataFrame by stacking the columns representing years into rows. This will allow to perform operations on the data more easily.

In [29]:
year_cols = df.columns[1:]

stack_expr = ", ".join([f"'{c}', `{c}`" for c in year_cols])
stack_str = f"stack({len(year_cols)}, {stack_expr}) as (Year, Nights)"

flat_df = df.select('Country', F.expr(stack_str))
flat_df = flat_df.filter(F.col('Nights').isNotNull())

flat_df.show()

+--------+----+--------+
| Country|Year|  Nights|
+--------+----+--------+
| Belgium|2006|16039090|
| Belgium|2007|16271311|
| Belgium|2008|16360702|
| Belgium|2009|15451017|
| Belgium|2010|16169676|
| Belgium|2011|16723867|
| Belgium|2012|16432646|
| Belgium|2013|16511721|
| Belgium|2014|17068872|
| Belgium|2015|18852087|
|Bulgaria|2006|11944694|
|Bulgaria|2007|12006786|
|Bulgaria|2008|11791454|
|Bulgaria|2009| 9460922|
|Bulgaria|2010|10547112|
|Bulgaria|2011|12426723|
|Bulgaria|2012|13451440|
|Bulgaria|2013|14370426|
|Bulgaria|2014|14077798|
|Bulgaria|2015|13352281|
+--------+----+--------+
only showing top 20 rows



## Queries

### Q1 - The average number of nights spent per country between 2007 and 2015

We'll calculate the average number of nights spent per country between 2007 and 2015.

In [30]:
avg_nights_df = flat_df \
    .filter(F.col('Year').between(2007, 2015)) \
    .groupBy('Country') \
    .agg(F.avg('Nights').alias('Average Nights')) \
    .orderBy(F.col('Average Nights').desc())

avg_nights_df = avg_nights_df.withColumn('Average Nights', F.format_number('Average Nights', 0))

avg_nights_df.show(100, False)

+--------------+--------------+
|Country       |Average Nights|
+--------------+--------------+
|Spain         |236,374,186   |
|Italy         |174,580,261   |
|France        |119,616,351   |
|Turkey        |94,525,217    |
|United Kingdom|87,177,340    |
|Austria       |75,411,503    |
|Germany       |64,505,370    |
|Greece        |62,349,340    |
|Croatia       |45,866,769    |
|Portugal      |30,078,644    |
|Netherlands   |29,345,543    |
|Czech Republic|20,607,822    |
|Belgium       |16,649,100    |
|Cyprus        |12,856,534    |
|Bulgaria      |12,387,216    |
|Ireland       |11,862,456    |
|Sweden        |11,620,353    |
|Poland        |11,387,049    |
|Hungary       |10,818,666    |
|Denmark       |9,591,138     |
|Norway        |8,065,221     |
|Malta         |7,676,956     |
|Montenegro    |7,064,597     |
|Finland       |5,457,530     |
|Slovenia      |5,373,349     |
|Slovakia      |4,251,239     |
|Estonia       |3,440,363     |
|Romania       |3,381,328     |
|Iceland

### Q2 - The years in which the number of nights spent in Greece was greater than the number of nights spent in 5 other countries

In [31]:
pivot_df = flat_df.groupBy('Year').pivot('Country').agg(F.first('Nights'))

for country in ['Poland', 'Romania', 'Germany', 'Sweden', 'Spain']:
    comparison_df = pivot_df.filter(F.col('Greece') > F.col(country))
    years = [row['Year'] for row in comparison_df.select('Year').collect()]

    print(f'Greece had more nights spent than {country} for {len(years)} years.')
    print('These years were: ', years)

Greece had more nights spent than Poland for 10 years.
These years were:  ['2012', '2014', '2013', '2009', '2006', '2011', '2008', '2007', '2015', '2010']
Greece had more nights spent than Romania for 10 years.
These years were:  ['2012', '2014', '2013', '2009', '2006', '2011', '2008', '2007', '2015', '2010']
Greece had more nights spent than Germany for 2 years.
These years were:  ['2009', '2011']
Greece had more nights spent than Sweden for 10 years.
These years were:  ['2012', '2014', '2013', '2009', '2006', '2011', '2008', '2007', '2015', '2010']
Greece had more nights spent than Spain for 0 years.
These years were:  []


### Q3 - The country with the highest number of nights spent for each year

In [32]:
window_spec = Window.partitionBy('Year').orderBy(F.desc('Nights'))

top_countries_df = flat_df.withColumn('rank', F.row_number().over(window_spec)) \
    .filter(F.col('rank') == 1) \
    .select('Year', 'Country', 'Nights')

top_countries_df = top_countries_df.withColumn('Nights', F.format_number('Nights', 0))

top_countries_df.show()

+----+-------+-----------+
|Year|Country|     Nights|
+----+-------+-----------+
|2006|  Spain|224,518,083|
|2007|  Spain|225,450,241|
|2008|  Spain|223,756,216|
|2009|  Spain|200,551,728|
|2010|  Spain|213,349,649|
|2011|  Spain|239,369,167|
|2012|  Spain|243,389,006|
|2013|  Spain|252,447,766|
|2014|  Spain|259,635,794|
|2015|  Spain|269,418,103|
+----+-------+-----------+



### Q4 - The years each country had the lowest number of nights among all countries.

In [33]:
window_spec = Window.partitionBy('Year').orderBy(F.col('Nights').asc())

ranked_df = flat_df.withColumn('Rank', F.row_number().over(window_spec))

min_nights_per_year = ranked_df.filter(F.col('rank') == 1).select('Year', 'Country', 'Nights')

result_df = min_nights_per_year.groupBy('Country').agg(F.collect_list('Year').alias('Years'))

result_df.show(truncate=False)

+-------------+------------------------------------------------------------+
|Country      |Years                                                       |
+-------------+------------------------------------------------------------+
|Liechtenstein|[2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]|
+-------------+------------------------------------------------------------+



## Stop

In [34]:
spark.stop()